### Bronze (raw) → Silver (cleaned) → Gold (aggregated)

- This is the **Medallion Architecture**, the industry standard for organizing data in a Lakehouse. 
- Instead of having one giant, messy pile of data, you move it through three stages of "refining".
- The Medallion Architecture is a data design pattern used to organize data as it flows through a system. 
- Think of it like a water filtration system: as water moves through different filters, it gets cleaner and safer to drink.
- In a data lakehouse, we move data through three distinct "buckets" to go from messy, raw files to clean, useful reports.

**1. Bronze (The Raw Layer)**
- This is the entry point for all your data. 
- You don't change anything here. 
- If the source data has errors, typos, or duplicates, you keep them exactly as they are. 
- This acts as your "Source of Truth"—if anything goes wrong later, you can always come back here to restart the process.

**2. Silver (The Validated Layer)**
- In this stage, you take your Bronze data and "clean" it. 
- You act like a filter to make the data high-quality. You perform tasks like:
- Removing duplicate records.
- Fixing "null" or missing values.
- Ensuring dates and numbers are in the correct format.
- Merging different tables together to provide a more complete picture.

**3. Gold (The Enriched Layer)**
- This is the final stage where the data is ready for the "business" to see. 
- Instead of millions of individual rows, Gold tables usually contain summaries. 
- The data here is organized to answer specific questions, like "What are the total sales per month?" or "Which region has the most active users?"

### Best practices for each layer

**1. Bronze Layer (Raw)**
- **The Golden Rule: Touch nothing.**
- **Keep it Immutable:** Never edit the data in this layer. It should be an exact "snapshot" of what came from the source.
- **Append Only:** Instead of updating rows, just keep adding new data to the end of the table.
- **Use Folders/Schema:** Organize your raw data by the date it arrived (e.g., year=2026/month=01/day=14).
- **Retain History:** Keep the raw data for as long as your budget allows. It’s your "safety net" if you need to rebuild everything from scratch.

**2. Silver Layer (Cleaned)**
- **The Golden Rule: Make it "Query-Ready."**
- **Enforce Schema:** Ensure that "Price" is always a number and "Date" is always a date. No more "garbage" data allowed.
- **Handle Duplicates:** This is where you use the MERGE command to ensure every record is unique.
- **Standardize:** If one source uses "M/F" for gender and another uses "Male/Female," pick one and stick to it here.
- **Data Masking:** If there is sensitive info (like phone numbers or emails), hide or encrypt them in this layer so only authorized people can see them.

**3. Gold Layer (Business)**
- **The Golden Rule: Make it fast and simple.**
- **Pre-calculate:** Instead of making the user calculate "Total Profit" every time, calculate it once and store it here.
- **Use Clear Names:** Use column names that a non-technical person understands (e.g., total_monthly_revenue instead of sum_rev_01).
- **Apply Optimization:** This is the best place to use ZORDER on columns that people use in filters (like Region or Category).
- **Keep it Lean:** Only include the columns needed for reports. Don't carry over technical IDs or "junk" columns from Silver.

**Summary of Best Practices**
| Feature | Bronze | Silver | Gold |
| ----- | ----- | ----- | ----- |
| Data Quality | Raw/Unverified | Cleaned/Verified | Highly Processed
| Format | Delta (or original) | Delta | Delta
| Storage Strategy | Append-only | Upsert (Merge) | Overwrite or Append
| Primary User | Data Engineers | Data Scientists | Business Analysts

### Incremental processing patterns

- Incremental processing means only moving the new or changed data from one layer to the next. 
- Instead of reprocessing millions of rows every day, you only touch the "delta" (the new stuff).

**1. The Append Pattern (Easiest)**
- This is used when your data only flows in one direction and never changes.
- **How it works:** You check the last time your pipeline ran, look for files created after that time, and add them to the next layer.
- **Best for:** Logs, sensor readings, or clickstream data where old records are never updated.

**2. The Upsert Pattern (Merge)**
- This is used when your source data can change (e.g., a customer changes their phone number).
- **How it works:** You use the MERGE command. It looks at the new batch of data:
- If the ID exists in Silver, it Updates the row.
- If the ID is brand new, it Inserts it.
- **Best for:** Customer profiles, inventory levels, or student grades.
- **Benefit:** It keeps your Silver/Gold layers perfectly in sync with the source without creating duplicates.

**3. Change Data Capture (CDC)**
- This is the most "pro" way to do incremental processing. 
- Some databases provide a special log that tells you exactly what changed: "Row 5 was deleted," "Row 10 was updated," "Row 12 was added."
- **How it works:** Delta Lake has a feature called Change Data Feed (CDF). It records the "before" and "after" for every single change.
- **Best for:** Complex systems where you need to know not just the current value, but how it changed over time.

**4. Structured Streaming (Auto-Pilot)**
- In Databricks, the most common way to handle all the patterns above is using Structured Streaming.
- The "Trigger Once" Trick: Even if you only want to run your code once a day (Batch), you can use the streaming API. It automatically keeps track of which files it has already processed using a "checkpoint" folder.
- How it looks in code:
```
(spark.readStream
  .format("cloudFiles") # Auto-loader
  .option("cloudFiles.format", "csv")
  .load(bronze_path)
  .writeStream
  .trigger(availableNow=True) # Process all new data then stop
  .checkpointLocation(checkpoint_path)
  .toTable("silver_table"))
```

**Why use Incremental Processing?**
- Cost: Processing 1,000 new rows is much cheaper than processing 10 billion old ones.
- Speed: Your "Gold" reports update in minutes instead of hours.
- Scalability: As your business grows, your pipeline stays fast because the "daily work" stays roughly the same size.

#### Task 1: Design 3-Layer Architecture

1. Bronze: ecommerce_bronze (Raw strings/as-is)
2. Silver: ecommerce_silver (Cleaned, typed, and deduplicated)
3. Gold: ecommerce_gold_brand_metrics (Aggregated for business)

#### Task 2: Build Bronze (Raw Ingestion)

In [0]:
# Define the path to your downloaded CSV
file_path = "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"

# Read the file with correct options
df = (spark.read
      .format("csv")
      .option("header", "true")        # Uses the first row as column names
      .option("inferSchema", "true")   # Automatically detects data types (e.g., price as double)
      .load(file_path))

In [0]:
# Verify the result
df.printSchema()
display(df.limit(5))

In [0]:
### save to bronze
df.write.format("delta").mode("append").saveAsTable("ecommerce_bronze")

#### Task 3: Build Silver (Cleaning & Validation)

In [0]:
from pyspark.sql.functions import col, split, coalesce, lit, to_timestamp

## Read from bronze
bronze_df = spark.table("ecommerce_bronze")

## convert string to timestamp
step1_df = bronze_df.withColumn("event_time", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss"))

In [0]:
## Handle null values
step2_df = step1_df.withColumn("brand", coalesce(col("brand"), lit("unknown")))

In [0]:
from pyspark.sql.functions import element_at 

## split the category code
step3_df = step2_df.withColumn("category_array", split(col("category_code"), r"\.")).withColumn("main_category", element_at(col("category_array"), 1)).withColumn("sub_category", element_at(col("category_array"), 2)).drop("category_array")

In [0]:
# Create the silver dataframe
silver_df = step3_df.dropDuplicates(["event_time", "user_id", "product_id"])

# Save to Silver Table
silver_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_silver")

### Task 4: Build Gold (Business Aggregates)

In [0]:
from pyspark.sql.functions import col, sum, count, round

## filter for purchases and aggregate
gold_df = silver_df.filter(col("event_type") == "purchase").groupby("brand","main_category").agg(
    count("*").alias("total_transactions"),
    round(sum("price"),2).alias("total_revenue")
)

## save the table
gold_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_gold_brand_metrics")

## display results
display(gold_df.orderBy(col("total_revenue").desc()))

In [0]:
%sql
SELECT count(*) FROM ecommerce_silver WHERE event_type = 'purchase';

In [0]:
%sql
SELECT count(*) 
FROM ecommerce_silver 
WHERE event_type LIKE '%purchase%';

In [0]:
%sql
DESCRIBE TABLE ecommerce_silver;